# 多制約ナップサック問題

0-1ナップサック問題でナップサック制約が複数になったもの

$$
\max \sum_{j=1}^n v_j x_j \\
\mathrm{subject \ to \ } \sum_{j=1}^n a_{ij} x_j \le b_i, \quad i = 1,\ldots,m \\
x_i \in \{0,1\},  \quad j = 1,\ldots, n
$$


## 解法

1. MIPソルバで解く ([テキスト](https://scmopt.github.io/opt100/77mkp.html)で紹介されている)
2. 動的計画法で解く
    - 通常のナップサック問題のDPを多次元に拡張して，計算量 $O(nb_1b_2\cdots b_m)$ のアルゴリズムが作れる．ここでは実装しない．
3. ヒューリスティック解法: 双対問題を使った貪欲法

良さそうなサーベイ論文: [The Multidimensional Knapsack Problem: Structure and Algorithms](https://inria.hal.science/hal-01224914/document)

## 問題インスタンス

- [ORLIB](https://people.brunel.ac.uk/~mastjjb/jeb/orlib/mknapinfo.html)
- [データセットまとめ](https://www.researchgate.net/publication/271198281_Benchmark_instances_for_the_Multidimensional_Knapsack_Problem)
    - ORLIBと重複あり
    - 以下のコードではこちらのデータセット `All-MKP-Instances` を同じディレクトリに配置して実行する
- これらのデータセットは古くて容易に解けるとされているが，より難しい問題は提案されているか？

In [1]:
from util import read_mkp

data = 'All-MKP-Instances/chubeas/OR5x100/OR5x100.dat'
mkps = read_mkp(data)
mkp = mkps[0]
v, a, b, _ = mkp
print(f'{v=}')
print(f'{a=}')
print(f'{b=}')

v=[504.0, 803.0, 667.0, 1103.0, 834.0, 585.0, 811.0, 856.0, 690.0, 832.0, 846.0, 813.0, 868.0, 793.0, 825.0, 1002.0, 860.0, 615.0, 540.0, 797.0, 616.0, 660.0, 707.0, 866.0, 647.0, 746.0, 1006.0, 608.0, 877.0, 900.0, 573.0, 788.0, 484.0, 853.0, 942.0, 630.0, 591.0, 630.0, 640.0, 1169.0, 932.0, 1034.0, 957.0, 798.0, 669.0, 625.0, 467.0, 1051.0, 552.0, 717.0, 654.0, 388.0, 559.0, 555.0, 1104.0, 783.0, 959.0, 668.0, 507.0, 855.0, 986.0, 831.0, 821.0, 825.0, 868.0, 852.0, 832.0, 828.0, 799.0, 686.0, 510.0, 671.0, 575.0, 740.0, 510.0, 675.0, 996.0, 636.0, 826.0, 1022.0, 1140.0, 654.0, 909.0, 799.0, 1162.0, 653.0, 814.0, 625.0, 599.0, 476.0, 767.0, 954.0, 906.0, 904.0, 649.0, 873.0, 565.0, 853.0, 1008.0, 632.0]
a=[[42, 41, 523, 215, 819, 551, 69, 193, 582, 375, 367, 478, 162, 898, 550, 553, 298, 577, 493, 183, 260, 224, 852, 394, 958, 282, 402, 604, 164, 308, 218, 61, 273, 772, 191, 117, 276, 877, 415, 873, 902, 465, 320, 870, 244, 781, 86, 622, 665, 155, 680, 101, 665, 227, 597, 354, 597, 79

## 実行例: MIPソルバ

In [2]:
from pyscipopt import Model, quicksum

n = len(v)
m = len(b)

J = list(range(n))
I = list(range(m))
model = Model("mkp")
x = {}
for j in J:
    x[j] = model.addVar(vtype="B", name=f"x({j})")

constr = {}
for i in I:
    # for j in J:  # テキストにあるこのループは不要そう
    constr[i,j] = model.addCons(quicksum(a[i][j] * x[j] for j in J) <= b[i], f"Capacity({i},{j})")

model.setObjective(quicksum(v[j] * x[j] for j in J), sense='maximize')

model.optimize()

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 5 upgd conss, 0 impls, 0 clqs
   (0.0s) probing: 51/100 (51.0%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (0.0s) probing aborted: 50/50 successive totally useless probings
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 100 variables (100 bin, 0 int, 0 impl, 0 cont) and 5 constraints
      5 constraints of type <knapsack>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00
transformed 1/1 origi

## 実行例: 貪欲法

双対問題を考える

- ナップサック問題の線形緩和

$$
\max \sum_j v_j x_j \\
\mathrm{s.t.} \sum_j a_{ij} x_j \le b_i, i=1,...,m \\
x_j \ge 0, j=1,...,n
$$

- その双対問題

$$
\min \sum_i b_i y_i \\
\mathrm{s.t.} \sum_i a_{ij} y_i \ge v_j, j=1,...,n \\
y_i \ge 0, i=1,...,m
$$

双対問題の解を $y^*$ とする．
各品物の**効率性** $e_j, \ j=1,2,\cdots,n$ を以下のように定義する．

$$
e_j = \frac{v_j}{\sum_i a_{ij} y^*_i}
$$

貪欲法のアルゴリズム: 品物を効率性 $e_j$ が大きい順にソートし，制約を満たす限り入れていく


In [3]:
import numpy as np

def objective_value(solution, profits):
    obj = sum([solution[i] * profits[i] for i in range(len(profits))])
    return obj

def is_feasible(solution, weights, capacities):
    for j in range(len(weights)):
        total_weight = sum(weights[j][i]*solution[i] for i in range(len(weights[0])))
        if total_weight > capacities[j]:
            return False
    return True

def greedy(values, weights, capacities, dual_weights):
    n = len(values)
    m = len(weights)
    efficiency = [values[i] / sum([dual_weights[j]*weights[j][i] for j in range(m)]) for i in range(n)]
    order = np.argsort(efficiency)

    solution = [0] * n
    for idx in reversed(order):
        solution[idx] = 1
        if not is_feasible(solution, weights, capacities):
            solution[idx] = 0

    return solution

In [4]:
# 双対問題を解く

model = Model("mkp_dual")
y = {}
for i in I:
    y[i] = model.addVar(vtype="C", name=f"y({i})")

constr = {}
for j in J:
    constr[j] = model.addCons(quicksum(a[i][j] * y[i] for i in I) >= v[j], f"Constraint {j}")

model.setObjective(quicksum(b[i] * y[i] for i in I), sense='minimize')

model.optimize()

feasible solution found by trivial heuristic after 0.0 seconds, objective value 6.372100e+09
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, exhaustive) 0 del vars, 16 del conss, 0 add conss, 10 chg bounds, 16 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 16 deleted constraints, 0 added constraints, 10 tightened bounds, 0 added holes, 16 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 5 variables (0 bin, 0 int, 0 impl, 5 cont) and 84 constraints
     84 constraints of type <linear>
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   

In [6]:
sol = model.getBestSol()
dual_weight = [
    sol[y_var] for y_var in y.values()
]

solution = greedy(v, a, b, dual_weight)

print(solution)
print(objective_value(solution, v))

[0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0]
22619.0
